In [1]:
import os
import sys
os.chdir('/Users/somanathnanda/baba/Covid-19/')

import pandas as pd
pd.options.display.max_columns=None
pd.set_option('display.max_rows', None)
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

import matplotlib
from plotly.offline import init_notebook_mode, iplot
import warnings

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
warnings.filterwarnings('ignore')
init_notebook_mode(connected=True)

import tensorflow as tf
#tf.get_logger().setLevel('INFO')
#tf.autograph.set_verbosity(2)


from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import Callback, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import time
start = time.process_time()

In [2]:
def plot_timeseries(x_axis,data,name):
    num_cols=data.shape[1]
    go_arr=[]
    for i in range(num_cols):
        go_arr.append(go.Scatter(x=x_axis, y=data.iloc[:,i],name=data.columns[i]))   
    fig=go.Figure(go_arr)
    #fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)','paper_bgcolor': 'rgba(0, 0, 1, 0)',})
    fig.update_layout(
    title=name,font=dict(
        family="Courier New, monospace",
        color="#7f7f6a"))
    fig.show()
def plot_lines(df, title='', annotations=None):
    common_kw = dict(x=df.index, mode='lines+markers')
    xaxis = dict(title='Time Steps')
    data=[]
    for c in df.columns:
        data.append(go.Scatter(y=df[c], name=c, **common_kw))
    #data = [go.Scatter(y=df[c], name=c, **common_kw) for c in df.columns]
    layout = dict(title=title, showlegend=True, annotations=annotations, xaxis=xaxis)
    fig=dict(data=data, layout=layout)
    iplot(fig,show_link=True)

In [3]:
confirmed_data_df=pd.read_csv('data/time_series_covid19_confirmed_global.csv')
#recovered_data_df=pd.read_csv('data/time_series_covid19_recovered_global.csv')
death_data_df=pd.read_csv('data/time_series_covid19_deaths_global.csv')

country_pred_df=pd.DataFrame()
model_performance_df=pd.DataFrame()
forecast_df=pd.DataFrame()

def get_time_series_data(country_name):
    grouped_confirmed_df=confirmed_data_df.groupby('Country/Region')
    #grouped_recovered_df=recovered_data_df.groupby('Country/Region')
    grouped_death_df=death_data_df.groupby('Country/Region')
    country_confirmed_df=grouped_confirmed_df.get_group(country_name)
    #country_recovered_df=grouped_recovered_df.get_group(country_name)
    country_death_df=grouped_death_df.get_group(country_name)
    cleaned_confirmed_df=country_confirmed_df.drop(columns=['Province/State','Lat','Long']).sum(axis=0,skipna=True).to_frame()[1:]
    #cleaned_recovered_df=country_recovered_df.drop(columns=['Province/State','Lat','Long']).sum(axis=0,skipna=True).to_frame()[1:]
    cleaned_death_df=country_death_df.drop(columns=['Province/State','Lat','Long']).sum(axis=0,skipna=True).to_frame()[1:]
    ts_df=pd.DataFrame(cleaned_confirmed_df.values,columns=['confirmed_count'])
    #ts_df['recovered_count']=cleaned_recovered_df.values
    ts_df['death_count']=cleaned_death_df.values
    ts_df.index=cleaned_confirmed_df.index
    return ts_df
def plot(ts_df,country_name):
    #plot_timeseries(x_axis=ts_df.index,data=ts_df[['confirmed_count','recovered_count','death_count']],name='count_comparison_'+country_name)
    plot_lines(ts_df,'count_comparison_'+country_name)

# split a univariate sequence into samples
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    """
    Frame a time series as a supervised learning dataset.
    Arguments:
        data: Sequence of observations as a list or NumPy array.
        n_in: Number of lag observations as input (X).
        n_out: Number of observations as output (y).
        dropnan: Boolean whether or not to drop rows with NaN values.
    Returns:
        Pandas DataFrame of series framed for supervised learning.
    """
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg
def build_model_and_predict(ts_df, country_name):
    tf.random.set_seed(1234)
    np.random.seed(1)
    time_steps = 5   
    rolling_days = 5
    n_test=2
    n_batch_size = 1
    X=series_to_supervised(ts_df[['confirmed_count']], n_in=time_steps, n_out=rolling_days, dropnan=True)
    train,test=X[:-n_test],X[-n_test:]
    X_train=train.iloc[:,:time_steps]
    Y_train = train.iloc[:,-rolling_days:]
    X_test=test.iloc[:,:time_steps]
    Y_test = test.iloc[:,-rolling_days:]
    
    X_train=X_train.values.astype('float32').reshape((X_train.shape[0], X_train.shape[1], 1))
    Y_train=Y_train.values.astype('float32').reshape((Y_train.shape[0], Y_train.shape[1], 1))
    X_test=X_test.values.astype('float32').reshape((X_test.shape[0], X_test.shape[1], 1))
    Y_test=Y_test.values.astype('float32').reshape((Y_test.shape[0], Y_test.shape[1], 1))
    
    c = [
        ModelCheckpoint(country_name+'.hdf5', save_best_only=True, monitor='val_loss', mode='min', verbose=2, save_freq=1),
        EarlyStopping(monitor='val_loss', min_delta=0, patience=500, verbose=2),
        ReduceLROnPlateau(monitor='val_loss',factor=0.2,patience=5,min_lr=0.00001)
    ]
    model = Sequential()
    model.add(LSTM(100, activation='relu', kernel_initializer='he_normal', input_shape=(time_steps,1)))
    model.add(Dense(50, activation='relu', kernel_initializer='he_normal'))
    model.add(Dense(50, activation='relu', kernel_initializer='he_normal'))
    model.add(Dense(rolling_days))
    # compile the model
    model.compile(optimizer='adam', loss='mse', metrics=['mae','mse','accuracy'])
    # fit the model
    history=model.fit(X_train, Y_train, epochs=2000, batch_size=n_batch_size, verbose=2,callbacks=c,validation_data=(X_test, Y_test),shuffle=False)
    # evaluate the model
    [mse, mae, loss, accuracy]= model.evaluate(X_test, Y_test, batch_size=n_batch_size, verbose=1)
    model_performance=pd.DataFrame({'Country':[country_name],'mae':np.array(mae),'mse':np.array(mse),
                                    'rmse':np.array(np.sqrt(mse)),'accuracy':np.array(accuracy)})
    row = np.asarray(ts_df[-time_steps:].confirmed_count.values.astype('float32')).reshape((1, time_steps, 1))
    yhat=model.predict(row)
    return yhat,model_performance

In [4]:
country_names=['Ireland','India']#,'Sweden','Germany','Australia','Denmark','Iran','Austria','Canada','Singapore','Japan','Portugal','Brazil','US','Norway','United Kingdom']

In [ ]:
for country_name in country_names:
    ts_df=get_time_series_data(country_name)
    
    remove_the_number_of_zeros=(ts_df.confirmed_count==0).astype(int).sum()  #remove the number of zeroes from the begining. OIt can not appear in the middle
    ts_without_zeros_df=ts_df[remove_the_number_of_zeros-1:]
    
    yhat, model_performance = build_model_and_predict(ts_without_zeros_df,country_name)
    model_performance_df=model_performance_df.append(model_performance)
    final_date=pd.to_datetime(ts_df[-1:].index)
    pred_df=pd.DataFrame()
    for i in range(yhat.shape[1]):
        pred_df['Country']=[country_name]
        pred_df['pred_date']=final_date+(pd.DateOffset(i+1))
        pred_df['predicted_value']=round(yhat[0][i])
        forecast_df = forecast_df.append(pred_df)
        

In [6]:
for country_name in country_names:
    ts_df=get_time_series_data(country_name)
    df_1=ts_df[['confirmed_count']]
    df_1['date']=pd.to_datetime(df_1.index)
    df_1['date']=df_1['date'].dt.strftime('%Y-%m-%d')
    df_1.reset_index(drop=True, inplace=True)
    df_1.index=df_1['date']
    
    grouped_country_df=forecast_df.groupby('Country').get_group(country_name)
    grouped_country_df['pred_date']=grouped_country_df['pred_date'].dt.strftime('%Y-%m-%d')
    grouped_country_df.index=grouped_country_df.pred_date
    df_2=grouped_country_df['predicted_value']
    df_all=pd.merge(df_1,df_2,how='outer', left_index=True,right_index=True)
    
    plot(df_all,country_name)
    

In [7]:
forecast_df

,Country,pred_date,predicted_value
0,Ireland,2020-04-01,5196.0
0,Ireland,2020-04-02,5319.0
0,Ireland,2020-04-03,14202.0
0,Ireland,2020-04-04,21510.0
0,Ireland,2020-04-05,19830.0
0,India,2020-04-01,1761.0
0,India,2020-04-02,2226.0
0,India,2020-04-03,2558.0
0,India,2020-04-04,3102.0
0,India,2020-04-05,3407.0


#### Compare the model accuracy for each country

In [8]:
model_performance_df.index=model_performance_df['Country']
model_performance_df=model_performance_df.drop(columns=['Country'])
plot_lines(model_performance_df[['accuracy','mae','rmse']])

In [9]:
print('Total Time taken : '+str((time.process_time() - start)/60)+' mins')

Total Time taken : 50.52879181666667 mins
